# Inversion technique

## Continuous distributions

### Exponential distribution

The cumulative distribution function of an exponential is
$$
1-e^{-\lambda x}
$$
We can easily compute the inverse distribution function by solving the following equation with respect to $x$
$$
u = 1-e^{-\lambda x}
$$
giving
$$
x = -\frac{1}{\lambda} \ln (1-u)
$$
We can therefore generate an Exponential draw with the function

In [1]:
generate_exponential(u::Float64, θ::Float64) = -θ*log(1-u)

generate_exponential (generic function with 1 method)

In [2]:
using Distributions

Compare this function with the quantile function in the package Distributions.

In [3]:
u = 0.2
θ = 2.0
d = Exponential(θ)
x = generate_exponential(u,θ)
y = quantile(d,u)
println("x =",x)
println("y =",y)

x =0.4462871026284194
y =0.44628710262841953


In [4]:
u = 0.99
x = generate_exponential(u,θ)
y = quantile(d,u)
println("x =",x)
println("y =",y)

x =9.21034037197618
y =9.21034037197618


The numbers are very close, but not exaclty the same. To obtain the same numbers, we redefine the generation function as

In [5]:
?log1p

search: log1p log10



```
log1p(x)
```

Accurate natural logarithm of `1+x`. Throws [`DomainError`](@ref) for [`Real`](@ref) arguments less than -1.

There is an experimental variant in the `Base.Math.JuliaLibm` module, which is typically faster and more accurate.

# Examples

```jldoctest
julia> log1p(-0.5)
-0.6931471805599453

julia> log1p(0)
0.0
```


In [6]:
generate_exponential(u::Float64, θ::Float64) = -θ*log1p(-u)

generate_exponential (generic function with 1 method)

In [8]:
u = 0.2
x = generate_exponential(u,θ)
y = quantile(d,u)
println("x =",x)
println("y =",y)

x =0.44628710262841953
y =0.44628710262841953


On http://www.johndcook.com/julia_rng.html we find the code

In [8]:
function rand_exponential(mean)
    if mean <= 0.0
        error("mean must be positive")
    end
    -mean*log(rand())
end

rand_exponential (generic function with 1 method)

This does not correspond to the quantile function! It can raises issues when used in combination with variance reduction techniques.

The standard library in Julia uses another approach.

In [9]:
?randexp

search: randexp randexp! RangeIndex rsearchindex CartesianIndex



```
randexp([rng=GLOBAL_RNG], [T=Float64], [dims...])
```

Generate a random number of type `T` according to the exponential distribution with scale 1. Optionally generate an array of such random numbers. The `Base` module currently provides an implementation for the types [`Float16`](@ref), [`Float32`](@ref), and [`Float64`](@ref) (the default).


From https://en.wikipedia.org/wiki/Ziggurat_algorithm, we learn that the "ziggurat algorithm is an algorithm for pseudo-random number sampling. Belonging to the class of rejection sampling algorithms, it relies on an underlying source of uniformly-distributed random numbers, typically from a pseudo-random number generator, as well as precomputed tables."

The algorithm can have performance issues with variance reduction techniques (especially quasi-Monte Carlo methods), and we will avoid it.

The ziggurat method is also used for the normal distribution. In addition to the previous remark, we note from https://en.wikipedia.org/wiki/Inverse_transform_sampling that "on the other hand, it is possible to approximate the quantile function of the normal distribution extremely accurately using moderate-degree polynomials, and in fact the method of doing this is fast enough that inversion sampling is now the default method for sampling from a normal distribution in the statistical package R."

Julia used an interpolation of the error function as basis for the normal distribution.

In [10]:
?randn

search: randn randn! sprandn randstring ZeroMeanDiagNormal



```
randn([rng=GLOBAL_RNG], [T=Float64], [dims...])
```

Generate a normally-distributed random number of type `T` with mean 0 and standard deviation 1. Optionally generate an array of normally-distributed random numbers. The `Base` module currently provides an implementation for the types [`Float16`](@ref), [`Float32`](@ref), and [`Float64`](@ref) (the default).


In [11]:
?erf

search: erf erfi erfc erfcx erfinv erfcinv OverflowError StackOverflowError



```
erf(x)
```

Compute the error function of `x`, defined by $\frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} dt$ for arbitrary complex `x`.


In [12]:
d = Normal(1,2)

Distributions.Normal{Float64}(μ=1.0, σ=2.0)

In [13]:
quantile(d,0.95)

4.289707253902943

## Quantile and inverse distribution

### Cumulative distribution function

The distribution function completely caracterizes a random variable.
$$F_{U(0,1)}(x) = P[U(0,1) \leq x] = \int_0^x 1dt = x$$
$$F_X(a) = P[X \leq a]$$

Let $Z = F_X(X)$. If $x \in [0,1]$, the cumulative distribution function of $Z$ is
$$
F_Z(x) = P[Z \leq x] = P[F_X(X) \leq x] = P[X \leq F_X^{-1}(x)]
= F_X(F_X^{-1}(x)) = x.
$$

### Quantile

For a continous distribution $X$, the $\alpha$-quantile is the value $x$ such that
$$
F_X(x) = \alpha
$$

Since the distribution is continuous, the distribution function is invertible and
$$ x = F_X^{-1}(\alpha)$$
Therefore, the inversion technique is equivalent to the computation of the quantile, and therefore, we can summarize the inversion technique with the method

In [16]:
rand_contdist(rng::AbstractRNG, Dist::Distribution) = quantile(Dist, rand(rng))

rand_contdist (generic function with 1 method)

The method quantile is defined for any distribution in the library Distribution. It is computed either explicitely, either using Newton's method to locate a zero of the function
$$
g(x) \overset{def}{=} F_X(x) - \alpha
$$
We can also define a version of the function using the default generator

In [17]:
rand_contdist(Dist::Distribution) = quantile(Dist, rand())

rand_contdist (generic function with 2 methods)

In [18]:
N = Normal()
MT = MersenneTwister(12345)
rand_contdist(MT, N)

0.15785351319354535

In [19]:
rand_contdist(N)

-0.4364273324777784

In [29]:
# @elapsed rand_contdist(N)

## Box-Muller

In [20]:
Nsim = 500000000

500000000

In [21]:
function bm_test(N)
    for i = 1:N
        r = rand(2)
        θ = 2*pi*r[1]
        r2 = -2*log(r[2])
        s = sqrt(r2)
        x = s*cos(θ)
        y = s*sin(θ)
    end
end

bm_test (generic function with 1 method)

In [22]:
@elapsed bm_test(Nsim)

100.612766787

In [23]:
function inv_test(nsim)
    N = Normal()
    for i = 1:2*nsim
        rand_contdist(N)
    end
end

inv_test (generic function with 1 method)

In [24]:
@elapsed inv_test(Nsim)

60.249058632